In [1]:
import tensorflow as tf
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.neighbors import KDTree
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsClassifier
import matplotlib.pyplot as plt
import math
from collections import defaultdict
import operator
import scipy as sp
from sklearn import preprocessing
import json
import pandas as pd
from scipy.stats import gamma
from math import log2

In [2]:
data_location_AT = ["/content/drive/MyDrive/DistanceClassifier/data/ATER",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATERDD",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATGEO",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATGEOGD",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATHGG",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATSF",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATSFDD",
                 "/content/drive/MyDrive/DistanceClassifier/data/ATSticky"]
                 
AT_original = "/content/drive/MyDrive/DistanceClassifier/data/ATOriginal"

In [3]:
#checking the sizes of all files
for i in data_location_AT:
  df = pd.read_csv(i, sep = ' ')
  print(df.shape)

(498, 30)
(499, 30)
(498, 30)
(499, 30)
(250, 30)
(498, 30)
(499, 30)
(498, 30)


In [62]:
x = np.arange(0,30,1)
y = ['class_ATERDD', 'class_ATGEO', 'class_ATGEOGD', 'class_ATHGG',
       'class_ATSF', 'class_ATSFDD', 'class_ATSticky']

In [4]:
orig_df = pd.DataFrame()
for protein in data_location_AT:
    df = pd.read_csv(protein, header = None, sep = ' ')
    orig_df = pd.concat([orig_df, df])
orig_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,35223.0,258905.0,64.0,1901966.0,631093.0,359.0,1438.0,0.0,0.0,1.395706e+07,6.950122e+06,1.149374e+06,1.061300e+04,10701.0,5.247000e+03,2156.0,10588.0,0.0,1.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34927.0,255662.0,56.0,1869481.0,623275.0,398.0,1283.0,0.0,0.0,1.364780e+07,6.828568e+06,1.138929e+06,9.912000e+03,9594.0,4.900000e+03,2022.0,11419.0,0.0,3.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34967.0,256552.0,65.0,1882504.0,626561.0,351.0,1441.0,0.0,0.0,1.379874e+07,6.894800e+06,1.143167e+06,1.057300e+04,10701.0,5.330000e+03,2106.0,10335.0,0.0,2.0,0.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35012.0,258223.0,66.0,1903187.0,637004.0,382.0,1427.0,0.0,0.0,1.401514e+07,7.038376e+06,1.182312e+06,1.028100e+04,10611.0,5.175000e+03,2166.0,11347.0,0.0,0.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35021.0,256701.0,69.0,1878882.0,626033.0,335.0,1518.0,0.0,0.0,1.374344e+07,6.860246e+06,1.142749e+06,1.109700e+04,11097.0,5.484000e+03,2100.0,9679.0,0.0,4.0,0.0,1.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,34891.0,2563234.0,35499.0,96233540.0,353716500.0,411779.0,21363704.0,958353.0,65804.0,2.276863e+09,1.110983e+10,6.968592e+10,2.350491e+09,336362636.0,5.756563e+09,5862285.0,204869723.0,635754333.0,28400605.0,133941806.0,1233072.0,29978358.0,16555206.0,54155646.0,22366646.0,1550739.0,7843467.0,541768.0,972085.0,136657.0
495,34670.0,2478617.0,34414.0,92953506.0,337510447.0,404706.0,20377996.0,906796.0,62903.0,2.204643e+09,1.058212e+10,6.657905e+10,2.202280e+09,324022406.0,5.462324e+09,5712218.0,198934133.0,591292316.0,27044852.0,126639245.0,1228172.0,28972748.0,15039316.0,51019563.0,20936894.0,1517345.0,7315878.0,521418.0,920226.0,129808.0
496,34570.0,2573062.0,35114.0,94960038.0,376499715.0,400521.0,21669174.0,948198.0,66130.0,2.198687e+09,1.130904e+10,7.869014e+10,2.341261e+09,330502911.0,6.215101e+09,5576114.0,201027945.0,652149348.0,29577267.0,130586483.0,1181830.0,29043618.0,16694314.0,55019060.0,22168112.0,1498737.0,7814422.0,522781.0,961399.0,146440.0
497,34456.0,2470787.0,34192.0,91868672.0,336377369.0,385589.0,20275015.0,901496.0,63582.0,2.136704e+09,1.056536e+10,6.505378e+10,2.209084e+09,318615792.0,5.371153e+09,5320778.0,190789333.0,590096749.0,26401456.0,125423859.0,1118297.0,27828350.0,15140592.0,51691962.0,20708736.0,1440749.0,7354702.0,502853.0,910327.0,134200.0


In [5]:
def clean_data(data_location_AT):
  df_comb = pd.DataFrame()
  for protein in data_location_AT:
    df = pd.read_csv(protein, header = None, sep = ' ')
    scaler = StandardScaler()
    df = pd.DataFrame(scaler.fit_transform(df))
    df['class'] = protein[47:]
    df_comb = pd.concat([df, df_comb])
  return df_comb

In [11]:
df = clean_data(data_location_AT)
df = pd.get_dummies(df, columns= ['class'], drop_first = False)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,class_ATER,class_ATERDD,class_ATGEO,class_ATGEOGD,class_ATHGG,class_ATSF,class_ATSFDD,class_ATSticky
0,0.378272,-0.628044,-0.867396,-1.174207,0.234298,-0.716365,-0.706826,-1.236299,-0.630634,-0.602595,-0.633824,0.635312,-1.481153,-0.852056,0.265619,0.045654,-1.110393,-0.874310,0.069078,-1.374876,-0.818990,-1.117916,-1.471380,-0.795314,-1.191009,-0.936952,-1.096026,-1.137285,-0.880395,-0.216162,0,0,0,0,0,0,0,1
1,-0.562627,0.161419,0.692086,0.171935,0.097232,-0.379422,0.403577,0.368224,0.869614,-0.207943,0.228592,0.029433,0.589996,0.263948,0.081971,-0.503979,-0.336294,0.109221,-0.070720,0.714385,-0.375042,-0.064792,-0.337639,0.892975,0.272568,0.470849,0.506649,0.843796,0.659010,0.814947,0,0,0,0,0,0,0,1
2,-1.409987,2.160838,-0.860816,0.163746,3.041619,-0.875497,0.846537,-0.646596,-1.808430,-0.868639,2.382832,3.034091,0.537981,-0.494814,2.211022,-0.922025,0.311763,0.957552,0.678127,-0.861447,-0.343900,-0.553609,0.183128,-1.185411,-0.312923,-1.133854,-1.234541,-0.932294,-1.655598,-1.995584,0,0,0,0,0,0,0,1
3,-1.833666,-0.664967,-0.140954,-1.265570,0.070013,-1.003197,-0.039797,0.048423,0.583515,-1.575126,-0.641531,0.215494,-0.526925,-0.802119,0.249322,-1.207991,-1.261631,0.172824,0.114468,-0.477970,-0.933254,-0.873918,0.103552,0.480801,-0.124985,-0.461137,0.363007,-0.348499,0.419127,0.962702,0,0,0,0,0,0,0,1
4,-0.397557,-0.310751,-1.304314,-0.650735,0.115240,-0.565734,-0.739338,-1.090228,-1.545384,-0.708970,-0.076840,0.206238,-0.910421,-0.834805,-0.025785,-0.654219,-0.252157,-0.545262,-0.165355,-1.285907,-0.331598,-0.676865,-0.339193,-1.471241,-0.903079,-1.148065,-1.242156,-1.373836,-1.383724,-1.312206,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,-1.071007,-1.038517,-0.290386,-1.064139,-1.002168,-0.047170,-0.241919,-0.721698,-0.044811,-1.095653,-1.060877,-0.947603,-0.323428,-0.354881,-0.176465,-0.567693,-0.346381,-0.692927,0.692778,-0.706453,0.502256,-1.756974,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0
495,-0.634973,-0.362154,-0.418054,-0.202613,-0.090388,-1.484208,-0.102031,2.194312,-0.044811,-0.115909,-0.002512,0.118113,0.173073,-0.176261,0.205085,-0.086037,-1.441284,1.330250,1.371945,1.898128,-1.074007,-0.395592,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0
496,2.869450,2.850024,-0.162719,2.849685,2.706508,0.745679,0.043452,-0.721698,-0.044811,2.855255,2.751193,2.420661,0.280175,0.135569,0.154211,2.283192,0.751723,-0.692927,0.692778,-0.706453,-1.074007,-0.735937,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0
497,-0.586525,-0.264804,0.475619,-0.206029,-0.099479,0.894338,0.306441,-0.721698,-0.044811,-0.174407,-0.115653,-0.114864,0.051785,0.323272,0.169756,-0.333374,0.945412,-0.692927,-0.665557,-0.706453,-0.285875,1.646481,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0


In [7]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,class_ATER,class_ATERDD,class_ATGEO,class_ATGEOGD,class_ATHGG,class_ATSF,class_ATSFDD,class_ATSticky
count,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3.747000e+03,3747.000000,3747.000000,3747.000000,3747.000000,3747.000000,3747.000000,3747.000000,3747.000000
mean,-8.114873e-15,-6.829116e-16,7.288005e-16,-2.144372e-16,3.199706e-16,2.479557e-16,6.823709e-17,1.576297e-17,1.743223e-16,2.149113e-16,-2.264965e-16,-1.523557e-16,-2.680891e-16,1.050371e-16,-4.152299e-16,4.241781e-16,-4.606966e-16,6.346671e-17,-1.713135e-16,-1.851261e-16,-1.488001e-16,-5.890375e-17,-2.191742e-16,6.686301e-17,-2.190224e-16,-2.000594e-16,-1.067853e-16,1.419705e-16,2.054816e-17,-9.894081e-17,0.133173,0.133440,0.133173,0.133440,0.066987,0.133173,0.133440,0.133173
std,8.564943e-01,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,8.944570e-01,1.000133e+00,9.311596e-01,1.000133e+00,1.000133e+00,1.000133e+00,9.311596e-01,9.311596e-01,9.311596e-01,9.311596e-01,0.339807,0.340095,0.339807,0.340095,0.250033,0.339807,0.340095,0.339807
min,-6.441301e+00,-3.384943e+00,-5.533073e+00,-3.301752e+00,-3.321211e+00,-3.172363e+00,-5.964539e+00,-3.094735e+00,-4.067485e+00,-3.072051e+00,-3.257026e+00,-3.217631e+00,-5.619240e+00,-5.539485e+00,-5.766471e+00,-2.945295e+00,-3.130495e+00,-3.182765e+00,-3.902196e+00,-3.339961e+00,-2.500330e+00,-3.477190e+00,-3.157945e+00,-3.965941e+00,-3.309385e+00,-2.749259e+00,-5.441214e+00,-2.986427e+00,-2.646776e+00,-3.583242e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-3.680253e-01,-6.842697e-01,-6.723488e-01,-6.285690e-01,-5.860221e-01,-6.108127e-01,-5.945541e-01,-7.216979e-01,-5.510370e-01,-6.051494e-01,-5.835329e-01,-4.858781e-01,-5.154712e-01,-5.635428e-01,-4.758155e-01,-6.022487e-01,-5.916014e-01,-6.852317e-01,-5.855790e-01,-7.064528e-01,-4.371686e-01,-5.521832e-01,-4.381520e-01,-4.550107e-01,-4.989268e-01,-5.295510e-01,-4.475763e-01,-4.668156e-01,-4.696021e-01,-4.604549e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,-1.397035e-01,-7.910086e-02,-1.537649e-01,-2.090622e-01,-1.958289e-01,-1.378578e-01,-2.478123e-01,-4.481107e-02,-1.949278e-01,-2.147477e-01,-2.345719e-01,-1.724539e-01,-1.404459e-01,-1.917838e-01,-2.193720e-01,-2.072289e-01,-2.561158e-01,-1.846383e-01,-2.548542e-01,-7.188961e-02,-1.846396e-01,-1.311679e-01,-4.481107e-02,-9.396565e-02,-7.777138e-02,-4.199967e-02,-9.309011e-02,-7.873414e-02,-1.923818e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.732534e-01,5.443090e-01,6.047026e-01,5.602496e-01,4.644416e-01,5.157424e-01,5.766289e-01,5.981221e-01,4.293308e-01,5.258057e-01,4.755762e-01,3.822342e-01,5.100290e-01,5.572378e-01,4.540641e-01,4.892441e-01,4.624369e-01,4.299615e-01,5.061558e-01,5.033988e-01,9.798694e-02,4.616573e-01,1.833388e-01,3.993075e-01,2.752269e-01,3.111206e-01,3.044108e-01,2.638576e-01,3.122883e-01,3.585083e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.133224e+00,5.503687e+00,4.971290e+00,6.856907e+00,8.822812e+00,7.136956e+00,7.315095e+00,9.456764e+00,2.231591e+01,8.946980e+00,9.027587e+00,1.368135e+01,9.621970e+00,9.126991e+00,1.209947e+01,9.544001e+00,9.340454e+00,9.643896e+00,9.276079e+00,9.356550e+00,1.127957e+01,9.552561e+00,1.342321e+01,2.231591e+01,1.285820e+01,1.285820e+01,9.421637e+00,1.391085e+01,1.476022e+01,9.941679e+00,1.000000,1.000000,1.000000,1.

In [52]:
def dataset_dicts(x, y):
  dataset = {}
  for y_col in y:
    dataset[y_col] = pd.concat([df[x], df[y_col]], axis = 1)
  return dataset

In [57]:
dataset = dataset_dicts(x,y)
dataset['class_ATERDD']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,class_ATERDD
0,0.378272,-0.628044,-0.867396,-1.174207,0.234298,-0.716365,-0.706826,-1.236299,-0.630634,-0.602595,-0.633824,0.635312,-1.481153,-0.852056,0.265619,0.045654,-1.110393,-0.874310,0.069078,-1.374876,-0.818990,-1.117916,-1.471380,-0.795314,-1.191009,-0.936952,-1.096026,-1.137285,-0.880395,-0.216162,0
1,-0.562627,0.161419,0.692086,0.171935,0.097232,-0.379422,0.403577,0.368224,0.869614,-0.207943,0.228592,0.029433,0.589996,0.263948,0.081971,-0.503979,-0.336294,0.109221,-0.070720,0.714385,-0.375042,-0.064792,-0.337639,0.892975,0.272568,0.470849,0.506649,0.843796,0.659010,0.814947,0
2,-1.409987,2.160838,-0.860816,0.163746,3.041619,-0.875497,0.846537,-0.646596,-1.808430,-0.868639,2.382832,3.034091,0.537981,-0.494814,2.211022,-0.922025,0.311763,0.957552,0.678127,-0.861447,-0.343900,-0.553609,0.183128,-1.185411,-0.312923,-1.133854,-1.234541,-0.932294,-1.655598,-1.995584,0
3,-1.833666,-0.664967,-0.140954,-1.265570,0.070013,-1.003197,-0.039797,0.048423,0.583515,-1.575126,-0.641531,0.215494,-0.526925,-0.802119,0.249322,-1.207991,-1.261631,0.172824,0.114468,-0.477970,-0.933254,-0.873918,0.103552,0.480801,-0.124985,-0.461137,0.363007,-0.348499,0.419127,0.962702,0
4,-0.397557,-0.310751,-1.304314,-0.650735,0.115240,-0.565734,-0.739338,-1.090228,-1.545384,-0.708970,-0.076840,0.206238,-0.910421,-0.834805,-0.025785,-0.654219,-0.252157,-0.545262,-0.165355,-1.285907,-0.331598,-0.676865,-0.339193,-1.471241,-0.903079,-1.148065,-1.242156,-1.373836,-1.383724,-1.312206,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,-1.071007,-1.038517,-0.290386,-1.064139,-1.002168,-0.047170,-0.241919,-0.721698,-0.044811,-1.095653,-1.060877,-0.947603,-0.323428,-0.354881,-0.176465,-0.567693,-0.346381,-0.692927,0.692778,-0.706453,0.502256,-1.756974,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,0
495,-0.634973,-0.362154,-0.418054,-0.202613,-0.090388,-1.484208,-0.102031,2.194312,-0.044811,-0.115909,-0.002512,0.118113,0.173073,-0.176261,0.205085,-0.086037,-1.441284,1.330250,1.371945,1.898128,-1.074007,-0.395592,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,0
496,2.869450,2.850024,-0.162719,2.849685,2.706508,0.745679,0.043452,-0.721698,-0.044811,2.855255,2.751193,2.420661,0.280175,0.135569,0.154211,2.283192,0.751723,-0.692927,0.692778,-0.706453,-1.074007,-0.735937,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,0
497,-0.586525,-0.264804,0.475619,-0.206029,-0.099479,0.894338,0.306441,-0.721698,-0.044811,-0.174407,-0.115653,-0.114864,0.051785,0.323272,0.169756,-0.333374,0.945412,-0.692927,-0.665557,-0.706453,-0.285875,1.646481,0.000000,-0.044811,-0.077771,-0.077771,0.000000,0.000000,0.000000,0.000000,0
